In [4]:
import os
import xml.etree.ElementTree as elemTree
import json

In [5]:
def get_br(gp):
    path = f"{fpath}/{gp}"
    brs = os.listdir(path)
    return brs

# BR 정보, SF (commit)정보
def openxml(xmldir):
    #print(xmldir)
    tree = elemTree.parse(xmldir)
    root = tree.getroot()
    assert len(root) == 2
    bug = root[0]
    commit = root[1]
    return bug,commit

def getbugtext(bug):
    summary = bug.find('summary').text
    description = bug.find('description').text
    return summary, description

def getbuginfo(bug):
    bid = bug.attrib['id']
    author = bug.attrib['author']
    openT = bug.attrib['open_date']
    closeT = bug.attrib['closed_time']
    return bid, author, openT, closeT

def getCtext(comment):
    return comment.text

def getCinfo(comment):
    Cid = int(comment.attrib['id'])
    Cauthor = comment.attrib['author']
    Cdate = comment.attrib['date']
    return Cid, Cauthor, Cdate

# return dictionary
def getCdict(bug):
    comments = list(bug.find('comments'))
    Cdict = {int(C.attrib['id']):{} for C in comments}
    for C in comments:
        Ctext = getCtext(C)
        Cid, Cauthor, Cdate = getCinfo(C)
        Cdict[Cid]['text'] = Ctext
        Cdict[Cid]['author'] = Cauthor
        Cdict[Cid]['date'] = Cdate
    return Cdict


In [8]:
# commits
def get_dmm(commit):
    dmm = commit.find('dmm_unit')
    NLOC = dmm.attrib['size']
    CCN = dmm.attrib['complexity']
    NPRAM = dmm.attrib['interfacing']
    return NLOC, CCN, NPRAM

def get_commitinfo(commit):
    commitID = commit.attrib['id']
    committer=commit.attrib['author']
    commitDate=commit.attrib['date']
    ccomplexity = commit.find('dmm_unit')
    dmm = commit.find('dmm_unit')
    cNLOC = dmm.attrib['size']
    cCCN = dmm.attrib['complexity']
    cNPRAM = dmm.attrib['interfacing']
    return commitID, committer, commitDate, cNLOC, cCCN, cNPRAM
    
def get_files(commit):
    files = [x for x in list(commit) if x.tag!='dmm_unit' ]
    return files

#fixed files
def get_fileinfo(file):
    type_=file.attrib['change_type']
    Ofname=file.attrib['old_name']
    Nfname=file.attrib['new_name']
    fcomplexity = file.find('file_info')
    fNLOC = fcomplexity.attrib['nloc']
    fCCN = fcomplexity.attrib['complexity']
    fNTOKEN = fcomplexity.attrib['token_count']
    # modified method 수
    children = [x.tag for x in list(file)]
    children = [x for x in children if x=='method']
    Nmm = len(children)
    return type_, Ofname, Nfname, fNLOC,fCCN,fNTOKEN, Nmm

#method 아닌거
def get_Nmethodinfo(file):
    aline=file.find('added_lines').text
    dline=file.find('deleted_lines').text

# method
def get_methodinfo(method):
    mName = method.attrib['name']
    mParams = method.attrib['parameters']
    aline = method.find('added_lines').text
    dline = method.find('deleted_lines').text
    return mName, mParams, aline, dline

def get_methodattribs(method):
    infos = method.find('method_info')
    nloc = infos.attrib['nloc']
    complexity=infos.attrib['complexity']
    toks=infos.attrib['token_count']
    nlevel=infos.attrib['nesting_level']
    sline=infos.attrib['start_line']
    eline = infos.attrib['end_line']
    return nloc, complexity, toks, nlevel, sline, eline

In [9]:
fpath = "./Generator/data/final_dataset/full/"
gps = os.listdir(fpath)
print(len(gps),'projects')

BRs = get_br(gps[0])
print(len(BRs), BRs[0])

56 projects
32 292.xml


In [11]:
# Complex version
for GroupProject in gps:
    BRs = get_br(GroupProject)
    print(f'[{gps.index(GroupProject)}/{len(gps)}]',GroupProject, len(BRs))
    for BR in BRs:
        finaljson = {'BR':{},'commit':{}}
        
        xmldir = f"{fpath}/{GroupProject}/{BR}"
        bug,commit = openxml(xmldir)
        # BR
        summary, description = getbugtext(bug)
        bid, author, openT, closeT = getbuginfo(bug)
        finaljson['BR']={'BR_id':bid,
                         'BR_author':author,
                         'BRopenT':openT,
                         'BRcloseT':closeT,
                         'BR_text':{'BRsummary':summary,
                                    'BRdescription':description
                                   },
                         'comments':{}
                        }

        #Comment
        comments = list(bug.find('comments'))
        cdict = {}
        for i,C in enumerate(comments):
            Ctext = getCtext(C)
            Cid, Cauthor, Cdate = getCinfo(C)

            key = f'comments_{i}'
            dict_ = {'comment_id':Cid,
                     'comment_author':Cauthor,
                     'commentT':Cdate,
                     'comment_text':Ctext
                    }
            
            cdict[key] = dict_
        finaljson['BR']['comments'] = cdict
            
              
        # commit
        commitID, committer, commitDate, cNLOC, cCCN, cNPRAM = get_commitinfo(commit)
        finaljson['commit']={'commit_id':commitID,
                             'commit_author':committer,
                             'commitT':commitDate,
                             'commit_complexity':{'commit_NLOC':cNLOC,
                                                  'commit_CCN':cCCN,
                                                  'commit_Nprams':cNPRAM
                                                 },
                             'changed_files':{}
                            }

        
        #file별로
        files = get_files(commit)
        fdict={}
        for i,file in enumerate(files):
            type_, Ofname, Nfname,fNLOC,fCCN,fNTOKEN,Nmm = get_fileinfo(file)
            key = f'file_{i}'
            fdict[key]={'file_change_type':type_,
                        'file_Nmethod':Nmm,
                        'file_old_name':Ofname,
                        'file_new_name':Nfname,
                        'file_complexity':{'file_NLOC':fNLOC,
                                           'file_CCN':fCCN,
                                           'file_NToken':fNTOKEN
                                          }
                       }

            
            if type_=='ADD' or type_=='DELETE' or type_=='RENAME':
                continue      
            
            # MODIFIED
            fdict[key]['hunks']={}
            if Nmm==0:
                hkey = f'hunk_0'
                modified_lines = file.find('modified_lines')
                aline=modified_lines.find('added_lines').text
                dline=modified_lines.find('deleted_lines').text
                fdict[key]['hunks'][hkey]={'Ismethod':0,
                                           'added_lines':aline,
                                           'deleted_lines':dline
                                          }
            else:
                methods = file.findall('method')
                for k,method in enumerate(methods):
                    hkey=f'hunk_{k}'
                    mName, mParams, aline, dline = get_methodinfo(method)
                    nloc, complexity, toks, nlevel, sline, eline = get_methodattribs(method)
                    fdict[key]['hunks'][hkey]={'Ismethod':1,
                                               'added_lines':aline,
                                               'deleted_lines':dline,
                                               'method_info':{'method_name':mName,
                                                              'method_params':mParams,
                                                              'method_startline':sline,
                                                              'method_endline':eline,
                                                              'method_complexity':{'method_NLOC':nloc,
                                                                                   'method_CCN':complexity,
                                                                                   'method_NToken':toks,
                                                                                   'method_nesting_level':nlevel
                                                                                  }
                                                             }
                                              }
            
        finaljson['commit']['changed_files']=fdict
        
        jsonpath = f'./JSonSet/fulljson/{GroupProject}/'
        if not os.path.isdir(jsonpath):
            os.makedirs(jsonpath)
        fname = BR.replace('.xml','.json')
        jsonpath = f'./JSonSet/fulljson/{GroupProject}/{fname}'
        with open(jsonpath,'w') as jf:
            json.dump(finaljson,jf)


[0/56] allenai+allennlp 32


AttributeError: module 'os' has no attribute 'mkdirs'

In [15]:
### simple version ######
for GroupProject in gps:
    BRs = get_br(GroupProject)
    print(f'[{gps.index(GroupProject)}/{len(gps)}]',GroupProject, len(BRs))
    for BR in BRs:
        finaljson = {'BR':{},'commit':{}}
        
        xmldir = f"{fpath}/{GroupProject}/{BR}"
        bug,commit = openxml(xmldir)
        # BR
        summary, description = getbugtext(bug)
        bid, author, openT, closeT = getbuginfo(bug)
        finaljson['BR']={'BR_id':bid,
                         'BR_author':author,
                         'BRopenT':openT,
                         'BRcloseT':closeT,
                         'BR_text':{'BRsummary':summary,
                                    'BRdescription':description
                                   },
                         'comments':{}
                        }

        #Comment
        comments = list(bug.find('comments'))
        cdict = {}
        for i,C in enumerate(comments):
            Ctext = getCtext(C)
            Cid, Cauthor, Cdate = getCinfo(C)

            key = f'comments_{i}'
            dict_ = {'comment_id':Cid,
                     'comment_author':Cauthor,
                     'commentT':Cdate,
                     'comment_text':Ctext
                    }
            
            cdict[key] = dict_
        finaljson['BR']['comments'] = cdict
            
              
        # commit
        commitID, committer, commitDate, cNLOC, cCCN, cNPRAM = get_commitinfo(commit)
        finaljson['commit']={'commit_id':commitID,
                             'commit_author':committer,
                             'commitT':commitDate,
                             'changed_files':{}
                            }

        
        #file별로
        files = get_files(commit)
        fdict={}
        for i,file in enumerate(files):
            type_, Ofname, Nfname,fNLOC,fCCN,fNTOKEN,Nmm = get_fileinfo(file)
            key = f'file_{i}'
            fdict[key]={'file_change_type':type_,
                        'file_Nmethod':Nmm,
                        'file_old_name':Ofname,
                        'file_new_name':Nfname
                       }

            
            if type_=='ADD' or type_=='DELETE' or type_=='RENAME':
                continue      
            
            # MODIFIED
            fdict[key]['hunks']={}
            if Nmm==0:
                hkey = f'hunk_0'
                modified_lines = file.find('modified_lines')
                aline=modified_lines.find('added_lines').text
                dline=modified_lines.find('deleted_lines').text
                fdict[key]['hunks'][hkey]={'Ismethod':0,
                                           'added_lines':aline,
                                           'deleted_lines':dline
                                          }
            else:
                methods = file.findall('method')
                for k,method in enumerate(methods):
                    hkey=f'hunk_{k}'
                    mName, mParams, aline, dline = get_methodinfo(method)
                    nloc, complexity, toks, nlevel, sline, eline = get_methodattribs(method)
                    fdict[key]['hunks'][hkey]={'Ismethod':1,
                                               'added_lines':aline,
                                               'deleted_lines':dline,
                                               'method_info':{'method_name':mName,
                                                              'method_params':mParams,
                                                              'method_startline':sline,
                                                              'method_endline':eline
                                                             }
                                              }
            
        finaljson['commit']['changed_files']=fdict
        
        jsonpath = f'./JSonSet/simplejson/{GroupProject}/'
        if not os.path.isdir(jsonpath):
            os.makedirs(jsonpath)
        fname = BR.replace('.xml','.json')
        jsonpath = f'./JSonSet/simplejson/{GroupProject}/{fname}'
        with open(jsonpath,'w') as jf:
            json.dump(finaljson,jf)

[0/53] allenai+allennlp 32
[1/53] alluxio+alluxio 32
[2/53] apache+incubator-mxnet 250
[3/53] apple+turicreate 196
[4/53] awslabs+gluon-ts 42
[5/53] bvlc+caffe 1
[6/53] carla-simulator+carla 20
[7/53] chainer+chainer 7
[8/53] cmu-perceptual-computing-lab+openpose 11
[9/53] cornellius-gp+gpytorch 13
[10/53] deepset-ai+farm 44
[11/53] dmlc+gluon-nlp 31
[12/53] eclipse+deeplearning4j 51
[13/53] espnet+espnet 8
[14/53] evilsocket+pwnagotchi 16
[15/53] explosion+spacy 174
[16/53] flairnlp+flair 6
[17/53] heartexlabs+label-studio 6
[18/53] horovod+horovod 48
[19/53] intel-isl+open3d 45
[20/53] jina-ai+jina 13
[21/53] keras-team+autokeras 23
[22/53] kubeflow+kubeflow 81
[23/53] microsoft+nni 50
[24/53] microsoft+onnxruntime 58
[25/53] microsoft+pai 45
[26/53] microsoft+recommenders 5
[27/53] mindsdb+mindsdb 9
[28/53] mlpack+mlpack 13
[29/53] mozilla+deepspeech 24
[30/53] nvidia+digits 15
[31/53] oneapi-src+onednn 42
[32/53] opencv+opencv 155
[33/53] openmined+pysyft 33
[34/53] openvinotoolkit